In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from visualization import *
from core.file import NuMLFile
from IPython.display import clear_output

In [3]:
fname = "/data/uboone/uboone_numu_slice_seq.h5"

In [4]:
"""Process all events in a file into graphs"""
f = NuMLFile(fname)
f.add_group("hit_table")
f.add_group("particle_table", ["g4_id", "parent_id", "type", "momentum", "start_process", "end_process"])
f.add_group("edep_table")
f.add_group("spacepoint_table")
f.read_data([0], [1000], use_seq=True, profile=False)
evt_list = f.build_evt(0, 1000)

In [ ]:
# Process single key
# planes = single_plane_graph_vis(evt_list[0])
# plot_event(handle_planes(planes), print_out=True, write=False)

In [25]:
print(evt_list[10]['particle_table'])

     g4_id  parent_id  type  momentum start_process           end_process
0        1          0    13  1.226558    b'primary'  b'FastScintillation'
1        2          0  2212  0.212783    b'primary'  b'FastScintillation'
2        3          0  2212  0.197717    b'primary'  b'FastScintillation'
3        4          0  2212  0.120768    b'primary'  b'FastScintillation'
4        5          0  2212  0.196712    b'primary'  b'FastScintillation'
..     ...        ...   ...       ...           ...                   ...
251    881        519    11  0.000341      b'compt'  b'FastScintillation'
252   1017        498    11  0.002596      b'compt'  b'FastScintillation'
253   1025        822    22  0.000069      b'eBrem'               b'phot'
254   1026        822    11  0.000625      b'eIoni'  b'FastScintillation'
255   1069       1025    11  0.000267       b'phot'  b'FastScintillation'

[256 rows x 6 columns]


In [16]:
# Visualize hierarchy
current_event = evt_list[3]
print(current_event)
planes = single_plane_graph_vis(current_event)
particle = current_event['particle_table']
hier = extract_hierarchy(planes, particle, current_event)
print(hier)

{'index': 4, 'hit_table':       global_plane  global_time  global_wire  hit_id   integral  local_plane  \
0                2  3124.469727       2415.0       0  70.382805            2   
1                2  1508.955811       2325.0       1  37.248268            2   
2                2  1443.360718       2319.0       2  97.944389            2   
3                2  1440.373291       2318.0       3  83.800812            2   
4                2  3618.190918       2282.0       4  29.601099            2   
...            ...          ...          ...     ...        ...          ...   
1638             0  1130.083618        293.0    1638  40.882240            0   
1639             0   101.192429        292.0    1639  71.620949            0   
1640             0   103.469971        291.0    1640  68.557381            0   
1641             0   270.996399        196.0    1641  61.386772            0   
1642             0   216.693497        196.0    1642  53.487831            0   

       local_

KeyError: False

In [ ]:
# Starting code for finding the particle hierarchy, assuming the above code works

# for each particle (given by g4_id), we have a list of their neighbors/children
# given in the form of hierarchy.iloc[g4_id]['neighbors']

# iterate through each particle and its children and create a big list of all the directed edges?
# seems inefficient...?

import networkx as nx
def vis_hierarchy(hier_df):
    ''' Visualize the hierarchy of a particle event '''
    # make a big list of all the edges
    edges = []
    for particle in hier_df.index.values.tolist():
        for neighbor in hier_df[particle]['neighbors']:
            edges.append((particle, neighbor))
    
    # turn this list into a networkx graph
    G = nx.Graph() # or G = nx.DiGraph()
    G.add_edges_from(edges)
    
    # visualize the graph
    # https://stackoverflow.com/questions/11479624/is-there-a-way-to-guarantee-hierarchical-output-from-networkx
    